# Objaverse-XL API Tutorial

Objaverse-XL is a Universe of 10M+ 3D Objects.

It is hosted on 🤗[Hugging Face](https://huggingface.co/datasets/allenai/objaverse-xl) and includes a [Python API on GitHub](https://github.com/allenai/objaverse-xl). This notebook provides a tutorial on downloading objects and annotations!

We'll get started by downloading the `objaverse` package from PyPi, which will allow us to easily download subsets of the dataset:

In [ ]:
!pip install objaverse --upgrade --quiet

import objaverse
objaverse.__version__

In [2]:
import objaverse
annotations2 = objaverse.load_lvis_annotations()
counts = [(k,len(v)) for k,v in annotations2.items()]

In [ ]:
# select N_categories objects
N_categories = 100
print(len(counts))
sum([v for k, v in counts])
collection = []
i = 0
for i, (k, v) in enumerate(annotations2.items()):
  if i >= N_categories:
    break
  collection.append(v[0])
print(collection)

In [ ]:
results = objaverse.load_objects(collection)

# Done!
The rest of the cells are from the old tutorial